# Keras: Classify Multi-Label

![houses](../images/iris.png)

Reference this great blog for machine learning cookbooks: [MachineLearningMastery.com "Multi-Label Classification"](https://machinelearningmastery.com/multi-label-classification-with-deep-learning/).

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import History

from sklearn.preprocessing import OneHotEncoder, PowerTransformer

import aiqc
from aiqc import datum

---

## Example Data

Reference [Example Datasets](example_datasets.ipynb) for more information.

In [10]:
df = datum.to_pandas('iris.tsv')

In [11]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [12]:
df.dtypes

sepal_length    float64
sepal_width     float64
petal_length    float64
petal_width     float64
species           int64
dtype: object

---

## a) High-Level API

Reference [High-Level API Docs](api_high_level.ipynb) for more information including how to work with non-tabular data.

In [13]:
splitset = aiqc.Pipeline.Tabular.make(
    dataFrame_or_filePath = df
    , label_column = 'species'
    , size_test = 0.22
    , size_validation = 0.12
    , label_encoder = OneHotEncoder(sparse=False)
    , feature_encoders = [{
        "sklearn_preprocess": PowerTransformer(method='box-cox', copy=False)
        , "dtypes": ['float64']
    }]
    
    , dtype = None
    , features_excluded = None
    , fold_count = None
    , bin_count = None
)


___/ featurecoder_index: 0 \_________

=> The column(s) below matched your filter(s) and were ran through a test-encoding successfully.
['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

=> Nice! Now all feature column(s) have encoder(s) associated with them.
No more Featurecoders can be added to this Encoderset.



In [14]:
def fn_build(features_shape, label_shape, **hp):
    model = Sequential()
    model.add(Dense(units=features_shape[0], activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(units=hp['neuron_count'], activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(units=label_shape[0], activation='softmax'))
    return model

In [15]:
def fn_train(model, loser, optimizer samples_train, samples_evaluate, **hp):
    model.compile(
        loss = loser
        , optimizer = optimizer
        , metrics = ['accuracy']
    )
    model.fit(
        samples_train["features"]
        , samples_train["labels"]
        , validation_data = (
            samples_evaluate["features"]
            , samples_evaluate["labels"]
        )
        , verbose = 0
        , batch_size = hp['batch_size']
        , epochs = hp['epoch_count']
        , callbacks=[History()]
    )
    return model

In [16]:
hyperparameters = {
    "neuron_count": [9, 12]
    , "batch_size": [3]
    , "learning_rate": [0.03, 0.05]
    , "epoch_count": [30, 60]
}

In [19]:
queue = aiqc.Experiment.make(
    library = "keras"
    , analysis_type = "classification_multi"
    , fn_build = fn_build
    , fn_train = fn_train
    , splitset_id = splitset.id
    , encoderset_id = splitset.encodersets[0]
    , repeat_count = 1
    , hide_test = False
    , hyperparameters = hyperparameters
    
    , fn_lose = None #automated
    , fn_optimize = None #automated
    , fn_predict = None #automated
    , foldset_id = None
)

In [20]:
queue.run_jobs()

🔮 Training Models 🔮: 100%|██████████████████████████████████████████| 8/8 [00:31<00:00,  3.97s/it]


For more information on visualization of performance metrics, reference the [Visualization & Metrics](visualization.html) documentation.

---

## b) Low-Level API

Reference [Low-Level API Docs](api_high_level.ipynb) for more information including how to work with non-tabular data, and defining an optimizer.

In [21]:
dataset = aiqc.Dataset.Tabular.from_pandas(df)

In [22]:
label_column = 'species'

In [23]:
label = dataset.make_label(columns=[label_column])

In [24]:
featureset = dataset.make_featureset(exclude_columns=[label_column])

In [25]:
splitset = featureset.make_splitset(
    label_id = label.id
    , size_test = 0.22
    , size_validation = 0.12
)

In [26]:
encoderset = splitset.make_encoderset()

In [27]:
labelcoder = encoderset.make_labelcoder(
    sklearn_preprocess = OneHotEncoder(sparse=False)
)

In [28]:
featurecoder_0 = encoderset.make_featurecoder(
    sklearn_preprocess = PowerTransformer(method='yeo-johnson', copy=False)
    , dtypes = ['float64']
)


___/ featurecoder_index: 0 \_________

=> The column(s) below matched your filter(s) and were ran through a test-encoding successfully.
['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

=> Nice! Now all feature column(s) have encoder(s) associated with them.
No more Featurecoders can be added to this Encoderset.



In [29]:
def fn_build(features_shape, label_shape, **hp):
    model = Sequential()
    model.add(Dense(units=features_shape[0], activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(units=hp['neuron_count'], activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(units=label_shape[0], activation='softmax'))
    return model

Optional, just included to show users how to specify their own optimizer.

In [1]:
def fn_optimize(**hp):
    optimizer = keras.optimizers.Adamax()
    return optimizer

In [36]:
def fn_train(model, loser, optimizer, samples_train, samples_evaluate, **hp):
    model.compile(
        loss = loser
        , optimizer = optimizer
        , metrics = ['accuracy']
    )
    model.fit(
        samples_train["features"]
        , samples_train["labels"]
        , validation_data = (
            samples_evaluate["features"]
            , samples_evaluate["labels"]
        )
        , verbose = 0
        , batch_size = hp['batch_size']
        , epochs = hp['epoch_count']
        , callbacks=[History()]
    )
    return model

In [37]:
algorithm = aiqc.Algorithm.make(
    library = "keras"
    , analysis_type = "classification_multi"
    , fn_build = fn_build
    , fn_train = fn_train
)

In [38]:
hyperparameters = {
    "neuron_count": [9, 12]
    , "batch_size": [3]
    , "learning_rate": [0.03, 0.05]
    , "epoch_count": [30, 60]
}

In [39]:
hyperparamset = algorithm.make_hyperparamset(
    hyperparameters = hyperparameters
)

In [40]:
queue = algorithm.make_queue(
    splitset_id = splitset.id
    , hyperparamset_id = hyperparamset.id
    , encoderset_id  = encoderset.id
    , repeat_count = 1
)

In [41]:
queue.run_jobs()

🔮 Training Models 🔮: 100%|██████████████████████████████████████████| 8/8 [00:29<00:00,  3.75s/it]


For more information on visualization of performance metrics, reference the [Visualization & Metrics](visualization.html) documentation.